# Fashion MNIST

In [81]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset , DataLoader

In [82]:
torch.manual_seed(42)

In [83]:
df = pd.read_csv('/content/fmnist_small.csv')

In [84]:
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1,0,0,0,0,0,0,0,0,0,...,69,12,0,0,0,0,0,0,0,0
5996,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5997,8,0,0,0,0,0,0,0,0,0,...,39,47,2,0,0,29,0,0,0,0
5998,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
labels = df['label'].values
features = df.drop('label', axis=1).values

In [86]:
features.shape

(6000, 784)

In [87]:
x_train , x_test , y_train , y_test = train_test_split(features , labels , test_size=0.2 , random_state=42)

In [88]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [89]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, features, labels) :
    self.features = torch.tensor(features, dtype = torch.float32)
    self.labels = torch.tensor(labels, dtype = torch.long)

  def __len__(self) :
    return self.features.shape[0]

  def __getitem__(self, idx) :
    return self.features[idx], self.labels[idx]

In [90]:
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

In [91]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [92]:
class FashionMNISTModel(nn.Module):
  def __init__(self, num_features):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.model(x)


In [93]:
learning_rate = 0.1
epochs = 100
model = FashionMNISTModel(x_train.shape[1])
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [94]:
for epoch in range(epochs) :
  total_loss = 0.0
  for batch_features, batch_labels in train_loader :
    outputs = model(batch_features)
    loss = loss_fn(outputs, batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  total_loss /= len(train_loader)
  print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss}')

Epoch 1/100, Loss: 1.3216368440786999
Epoch 2/100, Loss: 0.7793365436792373
Epoch 3/100, Loss: 0.6427524652083715
Epoch 4/100, Loss: 0.5751657336950302
Epoch 5/100, Loss: 0.528180201848348
Epoch 6/100, Loss: 0.49528750121593473
Epoch 7/100, Loss: 0.4602432996034622
Epoch 8/100, Loss: 0.4359459094206492
Epoch 9/100, Loss: 0.41829214423894884
Epoch 10/100, Loss: 0.3984092291692893
Epoch 11/100, Loss: 0.3850264506538709
Epoch 12/100, Loss: 0.3722685620188713
Epoch 13/100, Loss: 0.3477362078428268
Epoch 14/100, Loss: 0.3467242218057315
Epoch 15/100, Loss: 0.3138812760512034
Epoch 16/100, Loss: 0.3126461291313171
Epoch 17/100, Loss: 0.29552092105150224
Epoch 18/100, Loss: 0.2857150869568189
Epoch 19/100, Loss: 0.27084775785605114
Epoch 20/100, Loss: 0.26237100958824155
Epoch 21/100, Loss: 0.25756089876095456
Epoch 22/100, Loss: 0.24232648963729542
Epoch 23/100, Loss: 0.23957946779827277
Epoch 24/100, Loss: 0.2261324493587017
Epoch 25/100, Loss: 0.2233684351791938
Epoch 26/100, Loss: 0.21165

In [97]:
model.eval()
correct = 0
total = 0
with torch.no_grad() :
  for batch_features, batch_labels in test_loader :
    outputs = model(batch_features)
    _, predicted = torch.max(outputs, 1)
    total += batch_labels.shape[0]
    correct += (predicted == batch_labels).sum().item()


  print(f'Accuracy: {correct/total}')


Accuracy: 0.8325
